In [6]:
import geojson
import pandas
import json
from shapely.geometry import shape, Point
from collections import OrderedDict
import csv
import numpy as np 
import pyproj    
import shapely
import shapely.ops as ops
from shapely.geometry.polygon import Polygon
from functools import partial

In [2]:
path = '../../TrentoTreeMap/data/geo_data_trees.geojson'

In [10]:
with open(path, 'r') as data:
    loaded_json = geojson.load(data)
    features = loaded_json['features']
    tree_names = set()
    for feature in features:
        properties = feature['properties']
        tree_names.add(properties['Name'])
    
    result = {}
    for tree in tree_names:
        result[tree] = {'count': 0, 'total_canopy_cover': 0}
        
    for feature in features: 
        properties = feature['properties']
        result[properties['Name']]['count'] += 1
        result[properties['Name']]['total_canopy_cover'] += float(properties['Canopy Cover (m2)'])
    
    sorted_dictionary = OrderedDict(sorted(result.items(), key=lambda item: item[1]['count'], reverse=True))
    
    result = []
    for key, value in sorted_dictionary.items():
        elem = sorted_dictionary[key]
        elem['mean_canopy_cover'] = elem['total_canopy_cover'] / elem['count']
        #elem.pop('total_canopy_cover')
        result += [[key, elem['count'], elem['mean_canopy_cover']]]
        
    with open('data1.csv', 'w+') as dump:
        writer = csv.writer(dump)
        writer.writerow(['name', 'count', 'mean_canopy_cover'])
        for line in result:
            writer.writerow(line)

In [3]:
circoscrizioni_path = '../../TrentoTreeMap/data/circoscrizioni.json'

In [22]:
df = pandas.read_csv('data1.csv')
top_n_tree_names = list(df['name'][:10])

with open(circoscrizioni_path, 'r') as c:
    with open(path, 'r') as data:
        loaded_json = geojson.load(data)
        circoscrizioni = geojson.load(c)
        result = {'SARDAGNA': {'Other': 1}, 'MEANO': {}, 'ARGENTARIO': {}, 'BONDONE': {}, 'MATTARELLO': {}, 'OLTREFERSINA': {}, 'POVO': {}, 
                  'RAVINA-ROMAGNANO': {}, 'S.GIUSEPPE-S.CHIARA': {}, 'VILLAZZANO': {}, 'GARDOLO': {}, 
                  'CENTRO STORICO PIEDICASTELLO': {}}
        
        for f in loaded_json['features']:
            try:
                point = Point(f['geometry']['coordinates'])        
                for feature in circoscrizioni['features']:
                    polygon = shape(feature['geometry'])
                    if polygon.contains(point):
                        tree_name = f['properties']['Name']
                        circoscrizione = feature['properties']['nome']
                        try:
                            if tree_name in top_n_tree_names:
                                curr_count = result[circoscrizione][tree_name]
                                result[circoscrizione][tree_name] += 1
                            else:
                                curr_count = result[circoscrizione]['Other']
                                result[circoscrizione]['Other'] += 1
                        except KeyError:
                            if tree_name in top_n_tree_names:
                                result[circoscrizione][tree_name] = 1
                            else:
                                result[circoscrizione]['Other'] = 1
            except TypeError:
                continue
        
        with open('data2.csv', 'w+') as dump:
            tree_names = set()
            for feature in loaded_json['features']:
                properties = feature['properties']
                tree_names.add(properties['Name'])
                
            writer = csv.writer(dump)
            headers = ['circoscrizioni'] + top_n_tree_names + ['Other']
            
            print(result)
            
            writer.writerow(headers)
        
            for k, v in result.items():
                to_write = [k]
                for nome_albero in headers[1:]:
                    try:
                        to_write += [result[k][nome_albero]]
                    except KeyError:
                        to_write += [0]
                
                writer.writerow(to_write)

{'SARDAGNA': {'Other': 33, 'Cupressus': 6}, 'MEANO': {'Other': 145, 'Celtis australis': 9, 'Cupressus': 20, 'Platanus x hispanica': 9, 'Sophora japonica': 1, 'Carpinus betulus': 2, 'Tilia cordata': 1, 'Tilia x europaea': 19}, 'ARGENTARIO': {'Other': 270, 'Acer campestre': 34, 'Platanus x hispanica': 12, 'Sophora japonica': 5, 'Aesculus hippocastanum': 9, 'Tilia x europaea': 10, 'Celtis australis': 61, 'Tilia cordata': 5, 'Cupressus': 25, 'Prunus cerasifera': 5, 'Carpinus betulus': 18}, 'BONDONE': {'Other': 187, 'Acer campestre': 9, 'Tilia cordata': 9, 'Cupressus': 23, 'Carpinus betulus': 3, 'Sophora japonica': 7, 'Tilia x europaea': 28, 'Platanus x hispanica': 2, 'Aesculus hippocastanum': 21, 'Celtis australis': 1, 'Prunus cerasifera': 2}, 'MATTARELLO': {'Other': 160, 'Carpinus betulus': 13, 'Cupressus': 25, 'Celtis australis': 25, 'Tilia cordata': 6, 'Sophora japonica': 13, 'Platanus x hispanica': 6, 'Tilia x europaea': 5, 'Aesculus hippocastanum': 1, 'Prunus cerasifera': 8, 'Acer cam

In [18]:
df = pandas.read_csv('data1.csv')
top_n_tree_names = list(df['name'][:10])

with open(circoscrizioni_path, 'r') as c:
    with open(path, 'r') as data:
        loaded_json = geojson.load(data)
        circoscrizioni = geojson.load(c)
        result = {'SARDAGNA': {'Other': 1}, 'MEANO': {}, 'ARGENTARIO': {}, 'BONDONE': {}, 'MATTARELLO': {}, 'OLTREFERSINA': {}, 'POVO': {}, 
                  'RAVINA-ROMAGNANO': {}, 'S.GIUSEPPE-S.CHIARA': {}, 'VILLAZZANO': {}, 'GARDOLO': {}, 
                  'CENTRO STORICO PIEDICASTELLO': {}}
        
        circoscrizioni_list = result.keys()
        
        result = {}
        for tree in top_n_tree_names: 
            result[tree] = {}
        
        result['Other'] = {}
        
        for f in loaded_json['features']:
            try:
                point = Point(f['geometry']['coordinates'])        
                for feature in circoscrizioni['features']:
                    polygon = shape(feature['geometry'])
                    if polygon.contains(point):
                        tree_name = f['properties']['Name']
                        circoscrizione = feature['properties']['nome']
                        try:
                            if tree_name in top_n_tree_names:
                                curr_count = result[tree_name][circoscrizione]
                                result[tree_name][circoscrizione] += 1
                            else:
                                curr_count = result['Other'][circoscrizione]
                                result['Other'][circoscrizione] += 1
                        except KeyError:
                            if tree_name in top_n_tree_names:
                                result[tree_name][circoscrizione] = 1
                            else:
                                result['Other'][circoscrizione] = 1
            except TypeError:
                continue
        
        with open('data3.csv', 'w+') as dump:
            tree_names = set()
            for feature in loaded_json['features']:
                properties = feature['properties']
                tree_names.add(properties['Name'])
                
            writer = csv.writer(dump)
            headers = ['tree_name'] 
            for i in circoscrizioni_list:
                headers.append(i)
                
            writer.writerow(headers)
            
            print(result)
            
            for k, v in result.items():
                to_write = [k]
                for c in circoscrizioni_list:
                    try:
                        to_write += [result[k][c]]
                    except KeyError:
                        to_write += [0]
                writer.writerow(to_write)

            '''
            for k, v in result.items():
                to_write = [k]
                for nome_albero in headers[1:]:
                    try:
                        to_write += [result[nome_albero][k]]
                    except KeyError:
                        to_write += [0]
                
                writer.writerow(to_write)
            '''

{'Celtis australis': {'VILLAZZANO': 19, 'POVO': 21, 'MATTARELLO': 25, 'RAVINA-ROMAGNANO': 15, 'GARDOLO': 211, 'MEANO': 9, 'ARGENTARIO': 61, 'CENTRO STORICO PIEDICASTELLO': 242, 'OLTREFERSINA': 341, 'S.GIUSEPPE-S.CHIARA': 83, 'BONDONE': 1}, 'Aesculus hippocastanum': {'ARGENTARIO': 9, 'S.GIUSEPPE-S.CHIARA': 405, 'CENTRO STORICO PIEDICASTELLO': 91, 'OLTREFERSINA': 34, 'GARDOLO': 4, 'RAVINA-ROMAGNANO': 3, 'BONDONE': 21, 'POVO': 13, 'MATTARELLO': 1}, 'Carpinus betulus': {'BONDONE': 3, 'RAVINA-ROMAGNANO': 5, 'MATTARELLO': 13, 'VILLAZZANO': 33, 'GARDOLO': 53, 'CENTRO STORICO PIEDICASTELLO': 159, 'OLTREFERSINA': 126, 'S.GIUSEPPE-S.CHIARA': 123, 'ARGENTARIO': 18, 'MEANO': 2, 'POVO': 1}, 'Tilia cordata': {'VILLAZZANO': 35, 'RAVINA-ROMAGNANO': 34, 'POVO': 1, 'BONDONE': 9, 'MATTARELLO': 6, 'GARDOLO': 216, 'ARGENTARIO': 5, 'S.GIUSEPPE-S.CHIARA': 31, 'OLTREFERSINA': 116, 'CENTRO STORICO PIEDICASTELLO': 55, 'MEANO': 1}, 'Platanus x hispanica': {'POVO': 2, 'GARDOLO': 169, 'ARGENTARIO': 12, 'S.GIUSEPPE

In [16]:
with open(path, 'r') as data:
    loaded_json = geojson.load(data)
    features = loaded_json['features']
    
    result = []
    
    for feature in features: 
        properties = feature['properties']
        result.append([properties['Height (m)']])
    
    with open('data4.csv', 'w+') as dump:
        writer = csv.writer(dump)
        writer.writerow(['height'])
        for line in result:
            writer.writerow(line)

In [23]:
with open(path, 'r') as data:
    df = pandas.read_csv('data1.csv')
    top_n_tree_names = list(df['name'][:10])

    loaded_json = geojson.load(data)
    features = loaded_json['features']
    
    result = []
    for feature in features:
        properties = feature['properties']
        if properties['Name'] in top_n_tree_names:
            result.append([properties['Name'], properties['Height (m)'], properties['Gross Carbon Sequestration (kg/yr)'], properties['Canopy Cover (m2)']])

    with open('data5.csv', 'w+') as dump:
        writer = csv.writer(dump)
        writer.writerow(['name', 'height', 'co2_absorption', 'canopy_cover'])
        for line in result:
            writer.writerow(line)

In [28]:
# data 7 creation
with open('../assets/data7.csv', 'w+') as dump:
    df = pandas.read_csv('data2.csv')
    circ_names = list(df.circoscrizione)
    n_trees = list(df.sum(axis = 1))
    #print(circ_names)
    #print(list(df.sum(axis = 1)))
    result = []
    for i, circ in enumerate(circ_names):
        result.append([circ, circ, n_trees[i], 0])
    #print(result)

    writer = csv.writer(dump)
    writer.writerow(['name', 'code', 'pop', 'area'])
    for line in result:
        writer.writerow(line)

C:\Users\ricca\AppData\Local\Temp\ipykernel_12828\461144886.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  n_trees = list(df.sum(axis = 1))


In [34]:
df = pandas.read_csv('data1.csv')
top_n_tree_names = list(df['name'])

with open(circoscrizioni_path, 'r') as c:
    with open(path, 'r') as data:
        loaded_json = geojson.load(data)
        circoscrizioni = geojson.load(c)
        result = {'SARDAGNA': {}, 'MEANO': {}, 'ARGENTARIO': {}, 'BONDONE': {}, 'MATTARELLO': {}, 'OLTREFERSINA': {}, 'POVO': {}, 
                  'RAVINA-ROMAGNANO': {}, 'S.GIUSEPPE-S.CHIARA': {}, 'VILLAZZANO': {}, 'GARDOLO': {}, 
                  'CENTRO STORICO PIEDICASTELLO': {}}
        
        for f in loaded_json['features']:
            try:
                point = Point(f['geometry']['coordinates'])        
                for feature in circoscrizioni['features']:
                    polygon = shape(feature['geometry'])
                    circoscrizione = feature['properties']['nome']

                    if not 'area' in result[circoscrizione]:
                        geom_area = ops.transform(
                            partial(
                                pyproj.transform,
                                pyproj.Proj(init='EPSG:4326'),
                                pyproj.Proj(
                                    proj='aea',
                                    lat_1=polygon.bounds[1],
                                    lat_2=polygon.bounds[3]
                                )
                            ),
                        polygon)
                        result[circoscrizione]['area'] = geom_area.area
                        
                    if polygon.contains(point):
                        tree_name = f['properties']['Name']
                        
                        try:
                            result[circoscrizione]['total_canopy_cover'] += float(f['properties']['Canopy Cover (m2)'])
                        except KeyError:
                            result[circoscrizione]['total_canopy_cover'] = float(f['properties']['Canopy Cover (m2)'])
                                        
                        try:
                            result[circoscrizione]['oxygen_production'] += float(f['properties']['Oxygen Production (kg/yr)'])
                        except KeyError:
                            result[circoscrizione]['oxygen_production'] = float(f['properties']['Oxygen Production (kg/yr)'])
                        
                        try:
                            result[circoscrizione]['pop'] += 1
                        except KeyError:
                            if circoscrizione == 'SARDAGNA':
                                result[circoscrizione]['pop'] = 2
                            else:
                                result[circoscrizione]['pop'] = 1
            except TypeError:
                continue
                
        for k, v in result.items():
            result[k]['tree_density'] = result[k]['total_canopy_cover'] / result[k]['area']
        
        print(result)
        
        with open('data8.csv', 'w+') as dump:
            writer = csv.writer(dump)
            headers = ['circoscrizioni', 'code', 'area', 'oxygen_production', 'pop', 'tree_density']
            writer.writerow(headers)
            
            for k, v in result.items():
                to_write = [k, k]
                for k2, v2 in v.items():
                    if k2 != 'total_canopy_cover':
                        to_write += [v2]
                writer.writerow(to_write)

/Users/alessandro.arata/Workspace/miniconda3/envs/dv/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/alessandro.arata/Workspace/miniconda3/envs/dv/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/alessandro.arata/Workspace/miniconda3/envs/dv/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<co

{'SARDAGNA': {'area': 8664525.799191749, 'total_canopy_cover': 389.20000000000016, 'oxygen_production': 580.5, 'pop': 39, 'tree_density': 4.491878828917629e-05}, 'MEANO': {'area': 15714224.619070109, 'total_canopy_cover': 3136.0999999999935, 'oxygen_production': 2961.1000000000004, 'pop': 206, 'tree_density': 0.0001995707759067003}, 'ARGENTARIO': {'area': 13201049.127449466, 'total_canopy_cover': 12416.10000000007, 'oxygen_production': 6848.299999999997, 'pop': 454, 'tree_density': 0.0009405388829424761}, 'BONDONE': {'area': 36956241.92896825, 'total_canopy_cover': 5979.600000000029, 'oxygen_production': 4559.900000000003, 'pop': 292, 'tree_density': 0.00016180216623468154}, 'MATTARELLO': {'area': 16399254.780715883, 'total_canopy_cover': 4995.4000000000215, 'oxygen_production': 3492.6, 'pop': 264, 'tree_density': 0.000304611402578743}, 'OLTREFERSINA': {'area': 6963001.854266091, 'total_canopy_cover': 87988.80000000297, 'oxygen_production': 43063.999999999745, 'pop': 2695, 'tree_densit

In [20]:
import geojson
import csv

with open('./geo_data_trees.geojson', 'r') as data:
    loaded_json = geojson.load(data)
    with open('data9.csv', 'w', newline='') as dump:
        writer = csv.writer(dump)
        headers = ['species', 'longitude', 'latitude']
        writer.writerow(headers)
                
        for tree in loaded_json.features:
            print(tree.properties["Name"])
            print(tree["geometry"]["coordinates"])
            writer.writerow([tree.properties["Name"], tree["geometry"]["coordinates"][0], tree["geometry"]["coordinates"][1]])
            #break

Acer pseudoplatanus
[11.072014, 46.051599]
Acer pseudoplatanus
[11.072108, 46.051541]
Acer pseudoplatanus
[11.071959, 46.051668]
Acer pseudoplatanus
[11.15017, 46.063778]
Cupressus
[11.07155, 46.052305]
Acer pseudoplatanus
[11.071766, 46.052002]
Acer pseudoplatanus
[11.071857, 46.051874]
Acer pseudoplatanus
[11.071672, 46.052135]
Acer pseudoplatanus
[11.071439, 46.052551]
Acer pseudoplatanus
[11.071471, 46.052469]
Acer platanoides
[11.071801, 46.051954]
Acer pseudoplatanus
[11.071499, 46.052404]
Acer platanoides
[11.071592, 46.052238]
Acer pseudoplatanus
[11.071935, 46.051749]
Acer pseudoplatanus
[11.072265, 46.051379]
Acer pseudoplatanus
[11.143452, 46.048201]
Sophora japonica
[11.145229, 46.051329]
Tilia cordata
[11.14572, 46.051508]
Tilia cordata
[11.145829, 46.051532]
Tilia cordata
[11.145324, 46.051393]
Tilia cordata
[11.105603, 46.043994]
Tilia cordata
[11.10575, 46.042766]
Tilia cordata
[11.105752, 46.042668]
Tilia cordata
[11.105752, 46.042578]
Tilia cordata
[11.13707, 46.0547]

TypeError: 'NoneType' object is not subscriptable